In [ ]:
import os

tree_dir = '/content/drive/MyDrive/Trees in Satellite Imagery/Trees'
no_tree_dir = '/content/drive/MyDrive/Trees in Satellite Imagery/NoTrees'

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import preprocess_input

# Define data generators
datagen = ImageDataGenerator(preprocessing_function=preprocess_input, validation_split=0.2)

train_generator = datagen.flow_from_directory(
    '/content/drive/MyDrive/Trees in Satellite Imagery',  # parent directory containing tree and no_tree subdirectories
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='training'
)

validation_generator = datagen.flow_from_directory(
    '/content/drive/MyDrive/Trees in Satellite Imagery',  # parent directory containing tree and no_tree subdirectories
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)


Found 8320 images belonging to 2 classes.
Found 2080 images belonging to 2 classes.


In [ ]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

# Load the VGG16 model without the top layer
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add a new top layer for binary classification
x = base_model.output
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = Dense(1, activation='sigmoid')(x)

# Define the new model
model = Model(inputs=base_model.input, outputs=x)

# Freeze the layers of the base model
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

print('Success')


58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Success


In [ ]:
# Train the model
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator
)


Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


260/260 ━━━━━━━━━━━━━━━━━━━━ 1854s 7s/step - accuracy: 0.9358 - loss: 0.7484 - val_accuracy: 0.7644 - val_loss: 1.5719
Epoch 2/10
260/260 ━━━━━━━━━━━━━━━━━━━━ 49s 186ms/step - accuracy: 0.9920 - loss: 0.0434 - val_accuracy: 0.6692 - val_loss: 7.7347
Epoch 3/10
260/260 ━━━━━━━━━━━━━━━━━━━━ 50s 189ms/step - accuracy: 0.9960 - loss: 0.0240 - val_accuracy: 0.6995 - val_loss: 4.9732
Epoch 4/10
260/260 ━━━━━━━━━━━━━━━━━━━━ 83s 191ms/step - accuracy: 0.9990 - loss: 0.0030 - val_accuracy: 0.6712 - val_loss: 9.0275
Epoch 5/10
260/260 ━━━━━━━━━━━━━━━━━━━━ 81s 187ms/step - accuracy: 0.9985 - loss: 0.0100 - val_accuracy: 0.6971 - val_loss: 7.1411
Epoch 6/10
260/260 ━━━━━━━━━━━━━━━━━━━━ 49s 184ms/step - accuracy: 0.9977 - loss: 0.0148 - val_accuracy: 0.7418 - val_loss: 4.3418
Epoch 7/10
260/260 ━━━━━━━━━━━━━━━━━━━━ 84s 190ms/step - accuracy: 0.9989 - loss: 0.0067 - val_accuracy: 0.6880 - val_loss: 7.0797
Epoch 8/10
260/260 ━━━━━━━━━━━━━━━━━━━━ 48s 182ms/step - accuracy: 0.9986 - loss: 0.0088 - val_

In [ ]:
# Evaluate the model
val_loss, val_accuracy = model.evaluate(validation_generator)
print(f'Validation Loss: {val_loss}')
print(f'Validation Accuracy: {val_accuracy}')

# Save the model
model.save('/content/drive/My Drive/path_to_save_model/tree_classification_model.h5')
